In [90]:
import sys
import os 
from pathlib import Path
import pandas as pd
import json

project_path = Path(os.path.dirname(os.path.realpath("__file__"))).parent

from scripts.utils import log_df

In [91]:
orginal_openngc_path = project_path / 'raw_data' / 'OpenNGC' / 'database_files' / 'NGC.csv'
orginal_openngc_addendum_path = project_path / 'raw_data' / 'OpenNGC' / 'database_files' / 'addendum.csv'

openngc_path = project_path / 'raw_data' / 'OpenNGC.csv'

openngc_types_path = project_path / 'raw_data' / 'OpenNGC_types.csv' 
mike_path = project_path / 'raw_data' / 'mike_camilleri_list.csv' 

dwarf_draft_path = project_path/'data'/'draft'/'openngc_15min_catalog_draft.csv'
dwarf_json_path = project_path/'data'/'openngc_15min_catalog.json'

constellation_path = project_path/'raw_data'/'constellations.csv'

## Convert and merge OpenNGC

In [3]:
ngc_df = pd.read_csv(orginal_openngc_path, sep=";", dtype=str)

ngc_df.shape

(13968, 32)

In [4]:
add_df = pd.read_csv(orginal_openngc_addendum_path, sep=";", dtype=str)
add_df.shape

(24, 32)

In [5]:
add_df.columns == ngc_df.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [429]:
df = pd.concat([ngc_df, add_df])
df.to_csv(openngc_path, index=False)


## Create Dwarf2 Draft Catalog

In [57]:
openngc_columns = [
    'Name', 'Type', 'RA', 'Dec', 'Const', 'MajAx', 'MinAx', 'V-Mag', 'SurfBr', 
    'Cstar V-Mag',
    'M', 'NGC', 'IC', 'Cstar Names', 'Common names'
]
mike_columns = [
    'Catalog', 
    'Name', 'Name (no leading zero)', 'Name (final)', 'Common Name', 'NGC/ID', 
    "Width (')", "Height (')", 'Mag', 'Type', 'RA', 'Dec'
]

In [58]:
ngc_df = pd.read_csv(openngc_path, usecols=openngc_columns, dtype={'M': pd.Int64Dtype()})
ngc_df = ngc_df[ngc_df['Type'] != 'Dup']

log_df(ngc_df)
# (13340, 15)

(13340, 15)


,Name,Type,RA,Dec,Const,MajAx,MinAx,V-Mag,SurfBr,Cstar V-Mag,M,NGC,IC,Cstar Names,Common names
0,IC0001,**,00:08:27.05,+27:43:03.6,Peg,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN
1,IC0002,G,00:11:00.88,-12:49:22.3,Cet,0.98,0.32,NaN,23.45,NaN,<NA>,NaN,NaN,NaN,NaN
2,IC0003,G,00:12:06.09,-00:24:54.8,Psc,0.93,0.67,NaN,23.50,NaN,<NA>,NaN,NaN,NaN,NaN
3,IC0004,G,00:13:26.94,+17:29:11.2,Peg,1.17,0.84,NaN,23.01,NaN,<NA>,NaN,NaN,NaN,NaN
4,IC0005,G,00:17:34.93,-09:32:36.1,Cet,0.99,0.66,NaN,23.40,NaN,<NA>,NaN,NaN,NaN,NaN


In [59]:
mike_df = pd.read_csv(mike_path, usecols=mike_columns)

log_df(mike_df)
# (180, 12)

(180, 12)


,Catalog,Name,Name (no leading zero),Name (final),NGC/ID,Type,Common Name,Mag,RA,Dec,Width ('),Height (')
0,Caldwell,C 102,C 102,C 102,I2602,Oc,Theta Car Cluster,1.9,10:43:12,-64.40,50.0,50.0
1,Caldwell,C 103,C 103,C 103,2070,Bn,Tarantula Nebula,,05:38:42,-69.10,40.0,25.0
2,Caldwell,C 106,C 106,C 106,104,Gc,47 Tucanae,4,00:24:06,-72.08,31.0,31.0
3,Caldwell,C 108,C 108,C 108,4372,Gc,NaN,7.8,12:25:48,-72.67,19.0,19.0
4,Caldwell,C 014,C 14,C 14,869/884,Oc,"Double Cluster, h & chi Persei",4.3,02:20:00,57.13,30.0,30.0


In [60]:
# get rid of typos
mike_df.loc[(mike_df['NGC/ID'] == '-'), 'NGC/ID']  = pd.NA
mike_df.loc[(mike_df['NGC/ID'] == '2237-9'), 'NGC/ID']  = '2237'

# change names of objects that OpenNGC marks as duplicates
mike_df.loc[(mike_df['NGC/ID'] == '869/884'), 'NGC/ID']  = pd.NA # C 014
mike_df.loc[(mike_df['NGC/ID'] == '6992/5'), 'NGC/ID']  = '6992' # C 033
mike_df.loc[(mike_df['NGC/ID'] == '2244'), 'NGC/ID']  = '2239' # C 050
mike_df.loc[(mike_df['NGC/ID'] == 'Sh2-155'), 'NGC/ID']  = pd.NA # C 009
mike_df.loc[(mike_df['NGC/ID'] == 'I2118'), 'Name (no leading zero)']  = 'NGC 1909' # IC 2118
mike_df.loc[(mike_df['NGC/ID'] == 'I2118'), 'NGC/ID']  = '1909' # IC 2118
mike_df.loc[(mike_df['NGC/ID'] == 'I2169'), 'Name (no leading zero)']  = 'IC 447' # IC 2169
mike_df.loc[(mike_df['NGC/ID'] == 'I2169'), 'NGC/ID']  = 'I0447' # IC 2169
mike_df.loc[(mike_df['NGC/ID'] == '2527'), 'Name (no leading zero)']  = 'NGC 2520' # NGC 2527
mike_df.loc[(mike_df['NGC/ID'] == '2527'), 'NGC/ID']  = '2520' # NGC 2527
mike_df.loc[(mike_df['NGC/ID'] == '6383'), 'Name (no leading zero)']  = 'NGC 6374' # NGC 6383
mike_df.loc[(mike_df['NGC/ID'] == '6383'), 'NGC/ID']  = '6374' # NGC 6383

# add IC columns
mike_df.loc[(mike_df['NGC/ID'].notna()) & (mike_df['NGC/ID'].str.startswith('I')), 'IC']  = mike_df['NGC/ID']
mike_df['IC'] = mike_df['IC'].str.replace('I', '')
mike_df['IC name'] = 'IC ' + mike_df['IC'].str.strip()

# add NGC columns
mike_df.loc[(mike_df['NGC/ID'].notna()) & (mike_df['NGC/ID'].str.match('^(?![I])')), 'NGC']  = mike_df['NGC/ID']
mike_df['NGC name'] = 'NGC ' + mike_df['NGC'].str.strip()

mike_df['Name normalized']  = mike_df['Name (no leading zero)']

# manually set name for Messier, IC and NGC objects
mike_df.loc[(mike_df['Name (final)'] == 'IC 4715 = M 24'), 'Name normalized']  = 'M 24'
mike_df.loc[(mike_df['Name (final)'] == 'IC 4725 = M 25'), 'Name normalized']  = 'M 25'
mike_df.loc[(mike_df['Name (final)'] == 'NGC 2682 = M 67'), 'Name normalized']  = 'M 67'
mike_df.loc[(mike_df['Name (final)'] == 'NGC 6611 = M 16'), 'Name normalized']  = 'M 16'
mike_df.loc[(mike_df['Name (final)'] == 'NGC 6618 = M 17'), 'Name normalized']  = 'M 17'
mike_df.loc[(mike_df['Catalog'] == 'Caldwell') & (mike_df['IC'].notna()), 'Name normalized']  = mike_df['IC name']
mike_df.loc[(mike_df['Catalog'] == 'Caldwell') & (mike_df['NGC'].notna()), 'Name normalized']  =  mike_df['NGC name']

mike_df['mike'] = True

log_df(mike_df)
# (180, 18)

(180, 18)


,Catalog,Name,Name (no leading zero),Name (final),NGC/ID,Type,Common Name,Mag,RA,Dec,Width ('),Height ('),IC,IC name,NGC,NGC name,Name normalized,mike
0,Caldwell,C 102,C 102,C 102,I2602,Oc,Theta Car Cluster,1.9,10:43:12,-64.40,50.0,50.0,2602,IC 2602,NaN,NaN,IC 2602,True
1,Caldwell,C 103,C 103,C 103,2070,Bn,Tarantula Nebula,,05:38:42,-69.10,40.0,25.0,NaN,NaN,2070,NGC 2070,NGC 2070,True
2,Caldwell,C 106,C 106,C 106,104,Gc,47 Tucanae,4,00:24:06,-72.08,31.0,31.0,NaN,NaN,104,NGC 104,NGC 104,True
3,Caldwell,C 108,C 108,C 108,4372,Gc,NaN,7.8,12:25:48,-72.67,19.0,19.0,NaN,NaN,4372,NGC 4372,NGC 4372,True
4,Caldwell,C 014,C 14,C 14,<NA>,Oc,"Double Cluster, h & chi Persei",4.3,02:20:00,57.13,30.0,30.0,NaN,NaN,NaN,NaN,C 14,True


In [61]:
df = ngc_df.copy()

# get catalog and number
df[['Name catalog','Name number (with zeros)' ]] = df['Name'].str.strip().str.extract('^([A-Za-z]+)([0-9 A-Z]+)', expand=True)
df['Name normalized'] = df['Name number (with zeros)'].str.extract('^0*([0-9 A-Z]+)')

# handle IC and Messier objects
df.loc[df['Name catalog'] == 'IC',  'IC name'] = 'IC ' + df['Name normalized']
df.loc[df['Name catalog'] == 'M',  'M name'] = 'M ' + df['Name normalized']

df['Name normalized'] = df['Name catalog'] + ' ' + df['Name normalized']

df['Name number'] = df['Name number (with zeros)'].str.extract('([0-9]+)')
df['Name number'] = df['Name number'].astype(int)

# If object is a Messier object, use Messier data as the name
df.loc[df['M'].notna(), 'Name normalized'] = 'M ' + df['M'].astype(str)
df.loc[df['M'].notna(),  'Name catalog'] = 'M' 
df.loc[df['M'].notna(),  'Name number'] = df['M']


log_df(df)
# (13340, 21)

(13340, 21)


,Name,Type,RA,Dec,Const,MajAx,MinAx,V-Mag,SurfBr,Cstar V-Mag,...,NGC,IC,Cstar Names,Common names,Name catalog,Name number (with zeros),Name normalized,IC name,M name,Name number
0,IC0001,**,00:08:27.05,+27:43:03.6,Peg,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,IC,0001,IC 1,IC 1,NaN,1
1,IC0002,G,00:11:00.88,-12:49:22.3,Cet,0.98,0.32,NaN,23.45,NaN,...,NaN,NaN,NaN,NaN,IC,0002,IC 2,IC 2,NaN,2
2,IC0003,G,00:12:06.09,-00:24:54.8,Psc,0.93,0.67,NaN,23.50,NaN,...,NaN,NaN,NaN,NaN,IC,0003,IC 3,IC 3,NaN,3
3,IC0004,G,00:13:26.94,+17:29:11.2,Peg,1.17,0.84,NaN,23.01,NaN,...,NaN,NaN,NaN,NaN,IC,0004,IC 4,IC 4,NaN,4
4,IC0005,G,00:17:34.93,-09:32:36.1,Cet,0.99,0.66,NaN,23.40,NaN,...,NaN,NaN,NaN,NaN,IC,0005,IC 5,IC 5,NaN,5


In [62]:
df[df['Name']=='IC0359A'][['Name', 'M',  'Name catalog', 'Name number (with zeros)', 'Name number']]

,Name,M,Name catalog,Name number (with zeros),Name number
371,IC0359A,<NA>,IC,0359A,359


In [63]:
df[df['Name']=='IC4715'][['Name', 'M',  'Name catalog','Name number (with zeros)', 'Name number']]

,Name,M,Name catalog,Name number (with zeros),Name number
4903,IC4715,24,M,4715,24


In [64]:
merge_df = df.merge(mike_df,
                    left_on='Name normalized', right_on='Name normalized',
                    how='outer', suffixes=[None, '_MIKE'])

merge_df['Name number'] = merge_df['Name number'].astype(pd.Int64Dtype())
log_df(merge_df)
# (13349, 38)

(13349, 38)


,Name,Type,RA,Dec,Const,MajAx,MinAx,V-Mag,SurfBr,Cstar V-Mag,...,Mag,RA_MIKE,Dec_MIKE,Width ('),Height ('),IC_MIKE,IC name_MIKE,NGC_MIKE,NGC name,mike
0,IC0001,**,00:08:27.05,+27:43:03.6,Peg,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,IC0002,G,00:11:00.88,-12:49:22.3,Cet,0.98,0.32,NaN,23.45,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,IC0003,G,00:12:06.09,-00:24:54.8,Psc,0.93,0.67,NaN,23.50,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,IC0004,G,00:13:26.94,+17:29:11.2,Peg,1.17,0.84,NaN,23.01,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,IC0005,G,00:17:34.93,-09:32:36.1,Cet,0.99,0.66,NaN,23.40,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
merge_df[merge_df['Name']=='IC4715'][['Name', 'Name number']]

,Name,Name number
4582,IC4715,24
4583,IC4715,24


In [66]:
filter_df = merge_df.copy()

limit = 15

draft_df = filter_df[(filter_df['MajAx'] >= limit) | (filter_df['MinAx'] >= limit) | (filter_df['mike'] == True)]
draft_df = draft_df[((draft_df['V-Mag'].notna()) & (filter_df['V-Mag'] < 10)) | draft_df['V-Mag'].isna()]

log_df(draft_df)
# (210, 38) 15 + mike
# (137, 38) 15 
# (270, 38) 10 + mike
# (226, 38) 10

(210, 38)


/var/folders/zw/0kb57jnn5nv9k3pgxsz86n7h0000gq/T/ipykernel_46020/27436484.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  draft_df = draft_df[((draft_df['V-Mag'].notna()) & (filter_df['V-Mag'] < 10)) | draft_df['V-Mag'].isna()]


,Name,Type,RA,Dec,Const,MajAx,MinAx,V-Mag,SurfBr,Cstar V-Mag,...,Mag,RA_MIKE,Dec_MIKE,Width ('),Height ('),IC_MIKE,IC name_MIKE,NGC_MIKE,NGC name,mike
333,IC0341,Neb,03:40:55.69,+21:57:36.7,Tau,134.90,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
334,IC0342,G,03:46:48.50,+68:05:46.9,Cam,19.77,18.79,NaN,24.85,NaN,...,9.2,03:46:48,68.10,18.0,17.0,342,IC 342,NaN,NaN,True
341,IC0349,RfN,03:46:20.11,+23:56:23.3,Tau,25.70,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
345,IC0353,Neb,03:53:01.07,+25:50:52.8,Tau,181.97,30.20,NaN,NaN,NaN,...,NaN,03:55:00,25.48,180.0,180.0,353,IC 353,NaN,NaN,True
346,IC0354,Neb,03:53:57.91,+23:08:49.2,Tau,128.82,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
draft_df.to_csv(dwarf_draft_path, index=False)

## Create Dwarf2  Catalog

In [169]:
draft_df = pd.read_csv(dwarf_draft_path, dtype={'Name number': pd.Int64Dtype()})
log_df(draft_df)

(210, 38)


,Name,Type,RA,Dec,Const,MajAx,MinAx,V-Mag,SurfBr,Cstar V-Mag,...,Mag,RA_MIKE,Dec_MIKE,Width ('),Height ('),IC_MIKE,IC name_MIKE,NGC_MIKE,NGC name,mike
0,IC0341,Neb,03:40:55.69,+21:57:36.7,Tau,134.90,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,IC0342,G,03:46:48.50,+68:05:46.9,Cam,19.77,18.79,NaN,24.85,NaN,...,9.2,03:46:48,68.10,18.0,17.0,342,IC 342,NaN,NaN,True
2,IC0349,RfN,03:46:20.11,+23:56:23.3,Tau,25.70,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,IC0353,Neb,03:53:01.07,+25:50:52.8,Tau,181.97,30.20,NaN,NaN,NaN,...,NaN,03:55:00,25.48,180.0,180.0,353,IC 353,NaN,NaN,True
4,IC0354,Neb,03:53:57.91,+23:08:49.2,Tau,128.82,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [170]:
type_df = pd.read_csv(openngc_types_path)
type_df.rename(columns={'name': 'Type Name', 'category': 'Type Category'}, inplace=True)

log_df(type_df)

(21, 3)


,code,Type Name,Type Category
0,*,Star,stars
1,**,Double star,stars
2,*Ass,Association of stars,stars
3,OCl,Open Cluster,clusters
4,GCl,Globular Cluster,clusters


In [171]:
constellation_pd = pd.read_csv(constellation_path, usecols=['Abbreviations IAU', 'name'])
constellation_pd.rename(columns={'name': 'Constellation'}, inplace=True)
log_df(constellation_pd)

(88, 2)


,Abbreviations IAU,Constellation
0,And,Andromeda
1,Ant,Antlia
2,Aps,Apus
3,Aqr,Aquarius
4,Aql,Aquila


In [172]:
draft_df.columns

Index(['Name', 'Type', 'RA', 'Dec', 'Const', 'MajAx', 'MinAx', 'V-Mag',
       'SurfBr', 'Cstar V-Mag', 'M', 'NGC', 'IC', 'Cstar Names',
       'Common names', 'Name catalog', 'Name number (with zeros)',
       'Name normalized', 'IC name', 'M name', 'Name number', 'Catalog',
       'Name_MIKE', 'Name (no leading zero)', 'Name (final)', 'NGC/ID',
       'Type_MIKE', 'Common Name', 'Mag', 'RA_MIKE', 'Dec_MIKE', 'Width (')',
       'Height (')', 'IC_MIKE', 'IC name_MIKE', 'NGC_MIKE', 'NGC name',
       'mike'],
      dtype='object')

In [173]:
tmp_df =  draft_df.copy()
tmp_df.dropna(subset=['Name'], inplace=True)

for index, row in tmp_df.iterrows():
    # create string with multiple names
    names = set([row['Name normalized'],
                 row['IC name'], row['M name'],
                 row['Name (no leading zero)'], row['IC name_MIKE'], row['NGC name']])
    names = [name for name in names if pd.notna(name)]
    names.sort()
    tmp_df.at[index, 'Names'] = ', '.join(names)

    # create string with multiple common names
    common_names = set([row['Common names'], row['Common Name']])
    common_names = [name for name in common_names if pd.notna(name)]
    common_names.sort()
    tmp_df.at[index, 'Common Names'] = ', '.join(common_names)

tmp_df = tmp_df[
    [
        'Name normalized', 'Names', 'Common Names', 
        'Type', 'RA', 'Dec', 'Const', 'MajAx', 'MinAx', 'V-Mag',  'SurfBr', 
        'Name catalog', 'Name number', "Width (')", "Height (')"
    ]
].copy()

tmp_df.rename(columns={'Name normalized': 'Name'},  inplace=True)

tmp_df = tmp_df.sort_values(['Name catalog', 'Name number'])

log_df(tmp_df)
# (209, 15)

(209, 15)


,Name,Names,Common Names,Type,RA,Dec,Const,MajAx,MinAx,V-Mag,SurfBr,Name catalog,Name number,Width ('),Height (')
200,C 9,C 9,Cave Nebula,HII,22:57:54.0,+62:31:06,Cep,50.0,30.0,NaN,NaN,C,9,50.0,10.0
201,C 14,C 14,"Double Cluster, h & chi Persei",*Ass,02:20:42.0,+57:08:15,Per,50.0,50.0,NaN,NaN,C,14,30.0,30.0
202,C 41,C 41,Hyades,OCl,04:26:54.0,+15:52:00,Tau,329.0,NaN,NaN,NaN,C,41,330.0,330.0
203,C 99,C 99,Coalsack Nebula,DrkN,12:31:19.0,-63:44:36,Cru,NaN,NaN,NaN,NaN,C,99,400.0,300.0
204,Cl 399,Cl 399,"Brocchi's Cluster, Al Sufi's Cluster, Coathang...",*Ass,19:25:24.0,+20:11:00,Vul,70.0,NaN,3.6,NaN,Cl,399,NaN,NaN


In [174]:
dups = tmp_df[tmp_df.duplicated(subset=['Name'])]['Name'].values

In [175]:
dup_df = tmp_df[tmp_df['Name'].isin(dups)][['Name','Names', 'Common Names', 'Type']]
dup_df

,Name,Names,Common Names,Type
7,IC 405,"C 31, IC 405",Flaming Star Nebula,Neb
8,IC 405,IC 405,"Flaming Star Nebula, Flaming Star nebula",Neb
29,IC 2391,"C 85, IC 2391","Omicron Vel Cluster, omi Vel Cluster",OCl
30,IC 2391,IC 2391,omi Vel Cluster,OCl
32,IC 2602,"C 102, IC 2602","Theta Car Cluster, tet Car Cluster",OCl
33,IC 2602,IC 2602,"Southern Pleiades, tet Car Cluster",OCl
46,M 24,"IC 4715, M 24",Small Sgr Star Cloud,*Ass
47,M 24,"IC 4715, M 24","Sagittarius Star Cloud, Small Sgr Star Cloud",*Ass
48,M 25,"IC 4725, M 25",M 25,OCl
49,M 25,"IC 4725, IC C4725, M 25",,OCl


In [176]:
dwarf_catalog_df = tmp_df.copy()

def get_index_by_name(name):
    try:
        return dup_df[dup_df['Names'] == name].index[0]
    except:
        print(name, 'not found')
 

def get_index_by_common_name(name):
    try:
        return dup_df[dup_df['Common Names'] == name].index[0]
    except:
        print(name, 'not found..')
 
names = ['IC 405', 'IC 2391', 'IC 2602', 'IC 4725, IC C4725, M 25', 'NGC 6960', 'NGC 6992']
indexes = [get_index_by_name(name) for name in names]

names = ['Small Sgr Star Cloud', 'M  67']
indexes2 = [get_index_by_common_name(name) for name in names]

dwarf_catalog_df.drop(indexes + indexes2, inplace=True)

dwarf_catalog_df.at[get_index_by_name('C 102, IC 2602'), 'Common Names'] = 'Theta Car Cluster, Southern Pleiades'
dwarf_catalog_df.at[get_index_by_name('IC 4725, M 25'), 'Common Names'] = ""

log_df(dwarf_catalog_df)
# (201, 15)

(201, 15)


,Name,Names,Common Names,Type,RA,Dec,Const,MajAx,MinAx,V-Mag,SurfBr,Name catalog,Name number,Width ('),Height (')
200,C 9,C 9,Cave Nebula,HII,22:57:54.0,+62:31:06,Cep,50.0,30.0,NaN,NaN,C,9,50.0,10.0
201,C 14,C 14,"Double Cluster, h & chi Persei",*Ass,02:20:42.0,+57:08:15,Per,50.0,50.0,NaN,NaN,C,14,30.0,30.0
202,C 41,C 41,Hyades,OCl,04:26:54.0,+15:52:00,Tau,329.0,NaN,NaN,NaN,C,41,330.0,330.0
203,C 99,C 99,Coalsack Nebula,DrkN,12:31:19.0,-63:44:36,Cru,NaN,NaN,NaN,NaN,C,99,400.0,300.0
204,Cl 399,Cl 399,"Brocchi's Cluster, Al Sufi's Cluster, Coathang...",*Ass,19:25:24.0,+20:11:00,Vul,70.0,NaN,3.6,NaN,Cl,399,NaN,NaN


In [177]:
dwarf_catalog_df[dwarf_catalog_df.duplicated(subset=['Name'])]['Name'].values

array([], dtype=object)

In [178]:
merge_df = dwarf_catalog_df.merge(type_df, left_on='Type', right_on='code', how='left')
merge_df = merge_df.merge(constellation_pd, left_on='Const', right_on='Abbreviations IAU', how='left')

log_df(merge_df)
# (201, 20)

(201, 20)


,Name,Names,Common Names,Type,RA,Dec,Const,MajAx,MinAx,V-Mag,SurfBr,Name catalog,Name number,Width ('),Height ('),code,Type Name,Type Category,Abbreviations IAU,Constellation
0,C 9,C 9,Cave Nebula,HII,22:57:54.0,+62:31:06,Cep,50.0,30.0,NaN,NaN,C,9,50.0,10.0,HII,HII Ionized region,nebulae,Cep,Cepheus
1,C 14,C 14,"Double Cluster, h & chi Persei",*Ass,02:20:42.0,+57:08:15,Per,50.0,50.0,NaN,NaN,C,14,30.0,30.0,*Ass,Association of stars,stars,Per,Perseus
2,C 41,C 41,Hyades,OCl,04:26:54.0,+15:52:00,Tau,329.0,NaN,NaN,NaN,C,41,330.0,330.0,OCl,Open Cluster,clusters,Tau,Taurus
3,C 99,C 99,Coalsack Nebula,DrkN,12:31:19.0,-63:44:36,Cru,NaN,NaN,NaN,NaN,C,99,400.0,300.0,DrkN,Dark Nebula,nebulae,Cru,Crux
4,Cl 399,Cl 399,"Brocchi's Cluster, Al Sufi's Cluster, Coathang...",*Ass,19:25:24.0,+20:11:00,Vul,70.0,NaN,3.6,NaN,Cl,399,NaN,NaN,*Ass,Association of stars,stars,Vul,Vulpecula


In [179]:
del merge_df['code']
del merge_df['Type']
del merge_df['Const']
del merge_df['Abbreviations IAU']

merge_df.rename(columns={
    'Name': 'Catalogue Entry',
    'Names':'Alternative Entries',
    'Common Names': 'Familiar Name',
    'RA': 'Right Ascension',
    'Dec': 'Declination',
    'MajAx': 'Major Axis',
    'MinAx': 'Minor Axis',
    'V-Mag': 'Magnitude',
    'SurfBr': 'Surface Brightness',
    'Type Name': 'Type'
}, inplace=True)

log_df(merge_df)
# (201, 16)

(201, 16)


,Catalogue Entry,Alternative Entries,Familiar Name,Right Ascension,Declination,Major Axis,Minor Axis,Magnitude,Surface Brightness,Name catalog,Name number,Width ('),Height ('),Type,Type Category,Constellation
0,C 9,C 9,Cave Nebula,22:57:54.0,+62:31:06,50.0,30.0,NaN,NaN,C,9,50.0,10.0,HII Ionized region,nebulae,Cepheus
1,C 14,C 14,"Double Cluster, h & chi Persei",02:20:42.0,+57:08:15,50.0,50.0,NaN,NaN,C,14,30.0,30.0,Association of stars,stars,Perseus
2,C 41,C 41,Hyades,04:26:54.0,+15:52:00,329.0,NaN,NaN,NaN,C,41,330.0,330.0,Open Cluster,clusters,Taurus
3,C 99,C 99,Coalsack Nebula,12:31:19.0,-63:44:36,NaN,NaN,NaN,NaN,C,99,400.0,300.0,Dark Nebula,nebulae,Crux
4,Cl 399,Cl 399,"Brocchi's Cluster, Al Sufi's Cluster, Coathang...",19:25:24.0,+20:11:00,70.0,NaN,3.6,NaN,Cl,399,NaN,NaN,Association of stars,stars,Vulpecula


In [180]:
merge_df.to_json(dwarf_json_path, orient='records')